In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 1. 데이터 로드 (싱싱한 상태로 시작)
train_data = pd.read_csv('../data/train.csv')

# 2. Title 추출 (정규표현식 SyntaxWarning 해결: 앞에 r을 붙임)
# 이름에서 'Mr.', 'Mrs.' 등을 정확히 뽑아냅니다.
train_data['Title'] = train_data['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)

# 3. Title 단순화 및 숫자로 매핑 (KeyError 방지를 위해 X 설정 전에 수행)
train_data['Title'] = train_data['Title'].replace(['Lady', 'Countess','Capt', 'Col', \
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train_data['Title'] = train_data['Title'].replace('Mlle', 'Miss')
train_data['Title'] = train_data['Title'].replace('Ms', 'Miss')
train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
train_data['Title'] = train_data['Title'].map(title_mapping).fillna(0)

# 4. 나머지 결측치 및 인코딩 처리
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])
train_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].median())

train_data['Sex'] = train_data['Sex'].map({'female': 1, 'male': 0})
train_data['Embarked'] = train_data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# 5. 파생 변수 생성
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1
train_data['IsAlone'] = (train_data['FamilySize'] == 1).astype(int)

# 6. 학습 데이터 구성 (Title이 포함됨)
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilySize', 'IsAlone', 'Title']
X = train_data[features]
y = train_data['Survived']

# 7. 분리 및 학습
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
rf_model.fit(X_train, y_train)

print(f"새로운 정확도: {rf_model.score(X_val, y_val) * 100:.2f}%")

# 1. 테스트 데이터 로드
test_data = pd.read_csv('../data/test.csv')

# 2. 테스트 데이터 전처리 (학습 데이터와 '완전히 동일한' 로직 적용)
test_data['Title'] = test_data['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
test_data['Title'] = test_data['Title'].replace(['Lady', 'Countess','Capt', 'Col', \
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
test_data['Title'] = test_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
test_data['Title'] = test_data['Title'].replace('Mme', 'Mrs')
test_data['Title'] = test_data['Title'].map(title_mapping).fillna(0)

test_data['Age'] = test_data['Age'].fillna(train_data['Age'].median())
test_data['Fare'] = test_data['Fare'].fillna(train_data['Fare'].median())
test_data['Embarked'] = test_data['Embarked'].fillna('S')

test_data['Sex'] = test_data['Sex'].map({'female': 1, 'male': 0})
test_data['Embarked'] = test_data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1
test_data['IsAlone'] = (test_data['FamilySize'] == 1).astype(int)

# 3. 예측에 사용할 컬럼만 추출
X_test = test_data[features]

# 4. 학습된 모델(rf_model)로 정답 예측
predictions = rf_model.predict(X_test)

# 5. 제출용 양식(gender_submission.csv)에 맞춰 파일 저장
submission = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": predictions
})

submission.to_csv('submission_rf.csv', index=False)
